# Imports

In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold


2023-12-05 19:22:54.027676: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 19:22:54.027718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 19:22:54.028832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 19:22:55.136337: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Building The Environment
- Random temperature 
- Ideal Temp. is 37-39  

Goal: Build an agent to optimize the shower temperature

In [2]:
class FarmEnv(Env):
    def __init__(self):
        # Actions we can take, change temperature, pH, and water individually
        self.action_space = Box(low=np.array([-1, -0.5, -5]), high=np.array([1, 0.5, 5]))
        # Observation space (temperature, pH, water)
        self.observation_space = Box(low=np.array([0, 0, 0]), high=np.array([100, 14, 100]))
        # Set start values
        self.state = np.array([38 + random.randint(-3, 3), 7 + random.uniform(-0.5, 0.5), 50 + random.uniform(-10, 10)])
        # Set day length
        self.day_length = 60

    def step(self, action):
        # Clip action values to be within action space
        action = np.clip(action, self.action_space.low, self.action_space.high)

        # Apply action
        self.state += action

        # Clip values to be within observation space
        self.state = np.clip(self.state, self.observation_space.low, self.observation_space.high)

        # Reduce length by 1 second
        self.day_length -= 1

        # Calculate reward
        temp_reward = 1 if 37 <= self.state[0] <= 39 else -1
        ph_reward = 1 if 6.5 <= self.state[1] <= 7.5 else -1
        water_reward = 1 if 45 <= self.state[2] <= 55 else -1
        reward = temp_reward + ph_reward + water_reward

        # Check if day is done
        done = self.day_length <= 0

        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass

    def reset(self):
        # Reset farm values
        self.state = np.array([38 + random.randint(-3, 3), 7 + random.uniform(-0.5, 0.5), 50 + random.uniform(-10, 10)])
        # Reset day time
        self.day_length = 60
        return self.state


In [3]:
env = FarmEnv()


/home/ahmed/Downloads/SIC Hackathon/ml-env/lib64/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


### Init Test

In [4]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


Episode:1 Score:-164
Episode:2 Score:-144
Episode:3 Score:-132
Episode:4 Score:-84
Episode:5 Score:-76


# Model

### Train

In [5]:
log_path = os.path.join('Training', 'Logs')
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)
save_path = os.path.join('Training', 'Saved Models')


/home/ahmed/Downloads/SIC Hackathon/ml-env/lib64/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
# stop_callback = StopTrainingOnRewardThreshold(60, 1)
eval_callback = EvalCallback(env, eval_freq=10000, verbose=1, best_model_save_path=save_path)


model.learn(total_timesteps=400000, callback=eval_callback)


Logging to Training/Logs/PPO_1


/home/ahmed/Downloads/SIC Hackathon/ml-env/lib64/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -88      |
| time/              |          |
|    fps             | 1468     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -81.3       |
| time/                   |             |
|    fps                  | 951         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007835183 |
|    clip_fraction        | 0.0788      |
|    clip_range           | 0.2         |
|    entropy_loss         | -4.26       |
|    explained_variance   | -0.0019     |
|    learning_rate        | 0.

/home/ahmed/Downloads/SIC Hackathon/ml-env/lib64/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60.4     |
|    ep_rew_mean     | -52.8    |
| time/              |          |
|    fps             | 746      |
|    iterations      | 5        |
|    time_elapsed    | 13       |
|    total_timesteps | 10240    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60.4         |
|    ep_rew_mean          | -52.3        |
| time/                   |              |
|    fps                  | 735          |
|    iterations           | 6            |
|    time_elapsed         | 16           |
|    total_timesteps      | 12288        |
| train/                  |              |
|    approx_kl            | 0.0033320626 |
|    clip_fraction        | 0.0395       |
|    clip_range           | 0.2          |
|    entropy_loss         | -4.2         |
|    explained_variance   | 4.24e-05

### Save

In [7]:
save_path = os.path.join('Training', 'Saved Models', 'PPO_model2')
model.save(save_path)


# Evaluate

In [8]:
load_path = os.path.join('Training', 'Saved Models', 'PPO_model2')
model = PPO.load(load_path, env)


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [9]:
evaluate_policy(model, env, n_eval_episodes=50)


(112.92, 48.61639229724887)

## Test 

In [12]:
episodes = 10
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))


Episode:1 Score:120
Episode:2 Score:60
Episode:3 Score:30
Episode:4 Score:128
Episode:5 Score:78
Episode:6 Score:6
Episode:7 Score:38
Episode:8 Score:56
Episode:9 Score:104
Episode:10 Score:86
